# Extraction using PyPDF2

In [1]:
#!pip install PyPDF2
#!pip install tqdm
import pandas as pd
import numpy as np
import tqdm
import re
import PyPDF2


# Functions

In [2]:
def text_extractor(filename,pagenum):
    pdf=open((filename+'.pdf'), mode='rb')
    pdf_document=PyPDF2.PdfFileReader(pdf)
    target_page = pdf_document.getPage(pagenum-1)
    data = target_page.extractText()
    return data
def num_check(s):
    return any(i.isdigit() for i in s)
def bed_extractor(filename, pagenum):
    ext_data = text_extractor(filename,pagenum)
    samples = re.findall(r'\w+', ext_data)
    num = 0
    for items in samples:
        if 'Total' in items:
            target = re.findall(r'\d+', items)
            num = (target[1][2:])
        if 'facilityName' in items:
            name = items[12:] + ' Upazila Health Complex'
    if num == 0:
        pagenum = pagenum+1
        ext_data = text_extractor(filename,pagenum)
        samples = re.findall(r'\w+', ext_data)
        for items in samples:
            if 'Total' in items:
                target = re.findall(r'\d+', items)
                num = (target[1][2:])
            if 'Source' in items:
                if num_check(items):
                     num = num + (items[:-6])
    
    return [name, int(num)]

# Extraction

In [21]:
# files = ['1','3','10','15','20', '458']
pagenum = 2
USC_data = pd.DataFrame(columns = ['Name', 'Beds'])
others=[]   
for ind in range(1,482):
    try:
        place = len(USC_data)
        USC_data.loc[place] = bed_extractor('D:/bd health/upazilla/'+str(ind),pagenum)
    except:
        others.append(ind)

print(USC_data)
print(others)

                                  Name Beds
0    Abhoynagar Upazila Health Complex   50
1     Adamdighi Upazila Health Complex   70
2      Aditmari Upazila Health Complex   50
3    Agailjhara Upazila Health Complex   50
4    Azmiriganj Upazila Health Complex   31
..                                 ...  ...
461      Ulipur Upazila Health Complex   50
462    Ullapara Upazila Health Complex   81
463    Wazirpur Upazila Health Complex   50
464     Zanjira Upazila Health Complex   31
465    Zokiganj Upazila Health Complex   31

[466 rows x 2 columns]
[51, 53, 67, 80, 87, 90, 121, 125, 144, 161, 208, 221, 294, 307, 381]


In [23]:
USC_data.to_csv('D:/bd health/upazilla excel files/upazilla_bed_pypdf2.csv')